In [1]:
import torch
import pandas as pd
import numpy as np
import torch.utils as utils
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from tqdm.notebook import tqdm_notebook
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [2]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     # random.seed(seed)
     torch.backends.cudnn.deterministic = True
# setup_seed(20)

In [3]:
weight_agg = pd.read_csv('../../../train_data_revision1/aggregation-weight.txt',header=None).values

## 定义参数

In [4]:
num_1 = 12
# num_2 -> hid_1
num_2 = 8
num_3 = 16
# hid_2
hid_2 = 24
hid_dim_2 = 128
lstm_num_layer = 2
epochs = 500
drop_rate = 0.5
batch_size = 32
lr = 0.001
wdr = 0.0001
gcn_hid_dim = 8
hid_dim = 36
device = 'cuda'
n_his = 21
n_pred = 1
community_num = 139
step_size = 10
gamma = 0.996
file_path = '../../../train_data_revision1/'
data_path = file_path + 'cbg_input.npy'
cbg_edge_path = file_path + 'edge_21day.npy'
model_save_path = 'covid-21daysconv-weighted.pth'
model_read_path = ''
infect_data_path = file_path + 'number_of_infections_1day.csv'
cbg_to_com_file_path = file_path + 'cbg_to_comm.txt'
scaler = MinMaxScaler()
scaler_inf = MinMaxScaler()

## 定义函数

In [5]:
def totensor(data):
    return torch.Tensor(data).to(device)

In [6]:
def val(model, data_iter):
    model.eval()
    l_sum, n = 0.0, 0
    with torch.no_grad():
        for cbg_data,inf_data,edge_list,y in data_iter:
            b,c,_ = inf_data.size()
            y_pred = model(cbg_data,inf_data,edge_list).reshape(b,c)
            y = y.reshape(len(y),-1)
            l = loss(y_pred, y)
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        return l_sum / n

In [7]:
def evaluate_metric(model, data_iter):
    model.eval()
    with torch.no_grad():
        mae, sum_y, mape, mse = [], [], [], []
        for cbg_data,inf_data,edge_list,y in data_iter:
            y = y.reshape(len(y),-1)
            y = scaler_inf.inverse_transform(y.cpu().numpy().reshape(-1,1)).reshape(-1)
            
            b,c,_ = inf_data.size()
            y_pred = model(cbg_data,inf_data,edge_list).reshape(b,c)
            
            y_pred = scaler_inf.inverse_transform(y_pred.view(-1,1).cpu().numpy()).reshape(-1)
            d = np.abs(y - y_pred)
            mae += d.tolist()
            sum_y += y.tolist()
            mse += (d ** 2).tolist()
        MAE = np.array(mae).mean()
        RMSE = np.sqrt(np.array(mse).mean())
        WMAPE = np.sum(np.array(mae)) / np.sum(np.array(sum_y))
        return MAE, RMSE, WMAPE

## 定义模型

In [8]:
cbg_to_com_file = pd.read_csv(cbg_to_com_file_path)
def cbg_to_com(tensor_1,file):
    n_row = tensor_1.shape[0]
    dim = tensor_1.shape[2]
    data_2 = torch.zeros((n_row,139,dim)).to(device)
    for i in range(file.shape[0]):
        index_cbg = file.iloc[i,0]
        index_com = file.iloc[i,1]
        data_2[:,index_com] += tensor_1[:,index_cbg] * weight_agg[index_cbg][0]
    return data_2.to(device)

In [9]:
class MLP(torch.nn.Module):
    def __init__(self, n_i, n_h, n_o):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(n_i, n_h)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(n_h, n_o)
    def forward(self, input):
        input = self.linear1(input)
        input = self.relu(input)
        input = F.dropout(input, p=drop_rate, training=self.training)
        input = self.linear2(input)
        return input

In [10]:
class GCN(torch.nn.Module):
    def __init__(self,in_dim,out_dim,hid_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_dim, hid_dim,add_self_loops=True)
        self.conv2 = GCNConv(hid_dim, out_dim,add_self_loops=True)
    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=drop_rate, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, p=drop_rate, training=self.training)
        return x

In [11]:
class Covid_gcn_pre(nn.Module):
    def __init__(self):
        super(Covid_gcn_pre,self).__init__()
        self.input_mlp = MLP(3,hid_dim,num_1)
        self.cbg_gcn = GCN(num_1,num_2,gcn_hid_dim)
        self.inf_mlp = nn.Linear(1,num_3)
        self.hid_2_linear = nn.Linear(num_2+num_3,hid_2)
        self.output_mlp = MLP(hid_dim*n_his,hid_dim_2,1)
        self.rnn = nn.LSTM(hid_2, hid_dim, lstm_num_layer, batch_first = True)
    def forward(self,cbg,inf,edge):
        b,c,w,f = cbg.size()
        cbg = self.input_mlp(cbg)
        cbg_output = torch.Tensor().to(device)
        for j in range(b):
            cbg_edge_index = edge[j][0:2].long()
            cbg_edge_weight = edge[j][2]
            cbg_output_1 = torch.Tensor().to(device)
            for i in range(n_his):
                cbg_tmp = cbg[j,:,i,:].reshape(1,c,-1)
                cbg_tmp = self.cbg_gcn(cbg_tmp,cbg_edge_index,cbg_edge_weight).reshape(1,c,1,-1)
                cbg_output_1 = torch.cat([cbg_output_1,cbg_tmp],2)
            cbg_output = torch.cat([cbg_output,cbg_output_1],0)
        cbg_output = cbg_output.reshape(b,c,-1)
        cbg_output = cbg_to_com(cbg_output,cbg_to_com_file).reshape(-1,n_his,num_2)
        inf = inf.reshape(-1,n_his,1)
        inf = self.inf_mlp(inf)
        temp = torch.cat([cbg_output,inf],axis=2)
        temp = self.hid_2_linear(temp)
        temp,_ = self.rnn(temp)
        temp = temp.reshape(-1,139,hid_dim*n_his)
        output = self.output_mlp(temp)
        return output

In [12]:
model = Covid_gcn_pre().to(device)

## 准备数据

In [13]:
train_rate,val_rate,test_rate = 0.5,0.2,0.3
len_record = 300
num = len_record - n_his
len_train = int(num*train_rate)
len_val = int(num*val_rate)
len_test = num - len_train - len_val

## 计算edge_index,edge_weight

In [14]:
edge = np.load(cbg_edge_path)
edge_train = totensor(edge[:len_train])
edge_val = totensor(edge[len_train:len_val+len_train])
edge_test = totensor(edge[len_val+len_train:])

### cbg

In [17]:
cbg_data = np.load(data_path)
weekly_pattern = cbg_data[:,:,0]
popularity = cbg_data[:,:,1]
vulnerability = cbg_data[:,:,2]
weekly_pattern = scaler.fit_transform(weekly_pattern.reshape(-1,1)).reshape(cbg_data.shape[0],cbg_data.shape[1])
popularity = scaler.fit_transform(popularity.reshape(-1,1)).reshape(cbg_data.shape[0],cbg_data.shape[1])
vulnerability = scaler.fit_transform(vulnerability.reshape(-1,1)).reshape(cbg_data.shape[0],cbg_data.shape[1])
temp_data = np.stack([weekly_pattern,popularity,vulnerability],axis=2)
n_vertex = cbg_data.shape[1]
len_record = len(cbg_data)
x = np.zeros([num, n_his, n_vertex, 3])
for i in range(num):
    head = i
    tail = i+n_his
    x[i] = temp_data[head: tail]
x = np.swapaxes(x,1,2)
x.shape

(286, 2688, 14, 3)

In [18]:
cbg_train = totensor(x[:len_train])
cbg_val = totensor(x[len_train:len_val+len_train])
cbg_test = totensor(x[len_val+len_train:])

### infection

In [19]:
infection_data = np.array(pd.read_csv(infect_data_path,index_col=0))
infection_data = scaler_inf.fit_transform(infection_data.reshape(-1,1)).reshape(-1,139)
X,Y = [],[]
num = len(infection_data)-n_his
for i in range(num):
    X.append(infection_data[i:i+n_his])
    Y.append(infection_data[i+n_his:i+n_his+n_pred])
X = np.array(X)
Y = np.array(Y)
X = np.swapaxes(X,1,2)
Y = np.swapaxes(Y,1,2)
print(X.shape,Y.shape)

(286, 139, 14) (286, 139, 1)


In [20]:
infection_train = totensor(X[:len_train])
infection_val = totensor(X[len_train:len_val+len_train])
infection_test = totensor(X[len_val+len_train:])
y_train = totensor(Y[:len_train])
y_val = totensor(Y[len_train:len_val+len_train])
y_test = totensor(Y[len_val+len_train:])

### dataloader

In [21]:
train_data = utils.data.TensorDataset(cbg_train,infection_train,edge_train,y_train)
val_data = utils.data.TensorDataset(cbg_val,infection_val,edge_val,y_val)
test_data = utils.data.TensorDataset(cbg_test,infection_test,edge_test,y_test)
train_iter = utils.data.DataLoader(dataset = train_data,batch_size=batch_size,shuffle=False)
val_iter = utils.data.DataLoader(dataset = val_data,batch_size=batch_size,shuffle=False)
test_iter = utils.data.DataLoader(dataset = test_data,batch_size=batch_size,shuffle=False)

In [22]:
len(train_data),len(val_data),len(test_data)

(143, 57, 86)

## 训练环境

In [23]:
loss = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(),lr=lr,weight_decay=wdr)
scheduler = optim.lr_scheduler.StepLR(optimizer,step_size=step_size,gamma=gamma)

In [24]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=20, verbose=False, delta=0 ,path = ''):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''
        Saves model when validation loss decrease.
        验证损失减少时保存模型。
        '''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        # torch.save(model.state_dict(), self.path)
        # torch.save(model, self.path)
        self.val_loss_min = val_loss

early_stopping = EarlyStopping(patience=8, verbose=True, path = model_save_path)

In [ ]:
min_val_loss = np.inf
for epoch in tqdm_notebook(range(1,epochs + 1)):
    l_sum, n = 0.0 , 0
    for cbg_data,inf_data,edge_list,y in train_iter:
        b,c,_ = inf_data.size()
        y_pred = model(cbg_data,inf_data,edge_list).reshape(b,c)
        y = y.reshape(len(y),community_num)
        l = loss(y_pred,y)
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        scheduler.step()
        l_sum += l.item() * y.shape[0]
        n += y.shape[0]
    val_loss = val(model, val_iter)
    gpu_mem_alloc = torch.cuda.max_memory_allocated() / 1000000 if torch.cuda.is_available() else 0
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        torch.save(model.state_dict(), model_save_path)
    print('Epoch: {:03d} | Lr: {:.20f} |Train loss: {:.8f} | Val loss: {:.8f} | GPU occupy: {:.8f} MiB '.\
          format(epoch, optimizer.param_groups[0]['lr'], l_sum / n, val_loss, gpu_mem_alloc))
    if epoch > 150:
        early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping.")
        break
print('\nTraining finished.\n')

## test

In [ ]:
model.load_state_dict(torch.load(model_save_path))
test_MAE, test_RMSE, test_WMAPE = evaluate_metric(model, test_iter)
print(f'MAE {test_MAE:.6f} | RMSE {test_RMSE:.6f} | WMAPE {test_WMAPE:.8f}')